In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import os

import keras
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from keras.models import load_model

from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

import random as rn
import datetime

In [2]:
# Reading de los modelos.

# Ruta donde están los modelos.

model_CDM = load_model(filepath = "../Versión 3 - actual/modelos/datasetCDM_2023_04_17_19_35")

model_GRD = load_model(filepath = "../Versión 3 - actual/modelos/datasetGRD_2023_04_17_20_53")

model_GRDT = load_model(filepath = "../Versión 3 - actual/modelos/datasetGRDT_2023_04_17_11_29")

model_SEV = load_model(filepath = "../Versión 3 - actual/modelos/datasetSEV_2023_04_17_22_09")

In [3]:
x = pd.read_csv("../../../Datos/Procesados/x_values.csv", dtype = int)
y_cdm = pd.read_csv("../../../Datos/Procesados/y_dummies_cdm.csv", dtype = int)
y_grdt = pd.read_csv("../../../Datos/Procesados/y_dummies_grdtipo.csv", dtype = int)
y_grd = pd.read_csv("../../../Datos/Procesados/y_dummies_grd.csv", dtype = int)
y_sev = pd.read_csv("../../../Datos/Procesados/y_dummies_sev.csv", dtype = int)

In [30]:
y = y_cdm
y_str = 'CDM'

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0, stratify = y)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state = 0, stratify = y_train)

In [32]:
y_pred = model_CDM.predict(x_test)

# score = model_GRDT.evaluate(x_test, y_test, verbose = 1, batch_size = 128)

11900/11900 [==============================] - 15s 1ms/step


In [33]:
y_test_np = y_test.to_numpy(dtype = np.float32)

In [34]:
y_pred.shape[1]

23

In [35]:
num_classes = y_pred.shape[1]

In [36]:
print(y_test_np[:, 0])

[0. 0. 0. ... 1. 0. 0.]


In [37]:
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_np[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [38]:
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(num_classes)]))
mean_tpr = np.zeros_like(all_fpr)

for i in range(num_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

mean_tpr /= num_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

In [39]:
lst = y_test.columns

In [40]:
# Función para crear y guardar la curva ROC de una clase específica
def save_roc_plot(fpr, tpr, roc_auc, class_index, class_name, output_path):
    plt.figure(figsize=(10, 10))
    plt.plot(fpr[class_index], tpr[class_index], label=f'Curva ROC ({class_name}, área = {roc_auc[class_index]:.2f})')
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Clasificación aleatoria', alpha=0.8)
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Tasa de falsos positivos')
    plt.ylabel('Tasa de verdaderos positivos')
    plt.title(f'Curva ROC {class_name}')
    plt.legend(loc="lower right")
    plt.savefig(output_path, dpi=300)
    plt.close()

In [41]:
# Iterar sobre todas las clases y guardar la curva ROC para cada clase
for i in range(num_classes):
    class_name = lst[i]
    path_ = f"../../../Visualización de datos/Curvas ROC/{y_str}"
    isExists = os.path.exists(path=path_)
    if not isExists:
        os.makedirs(path_)
    output_path = f"{path_}/Curva_ROC_clase_{class_name}.png"
    save_roc_plot(fpr, tpr, roc_auc, i, class_name, output_path)

In [42]:
# plt.figure(figsize=(10, 10))

# # Trazar la curva ROC para cada clase
# for i in range(num_classes):
#     plt.plot(fpr[i], tpr[i], label=f'Curva ROC (Clase {lst[i]}, área = {roc_auc[i]:.2f})')

# # Trazar la curva ROC promedio
# plt.plot(fpr["macro"], tpr["macro"], label=f'Curva ROC promedio (área = {roc_auc["macro"]:.2f})', color='navy', linestyle='--')

# # Trazar una línea diagonal para representar una clasificación aleatoria
# plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r', label='Clasificación aleatoria', alpha=0.8)

# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('Tasa de falsos positivos')
# plt.ylabel('Tasa de verdaderos positivos')
# plt.title('Curva ROC multiclase')
# plt.legend(loc="lower right")
# plt.show()